Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

Matplotlib is building the font cache; this may take a moment.


SDK version: 1.60.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-292931
aml-quickstarts-292931
westeurope
a0a76bad-11a1-4a2d-9887-97a29122c8ed


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [4]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-bank-exp'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-bank-exp,quick-starts-ws-292931,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automlcluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [6]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "bankmarketing"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [7]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= False,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [9]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [10]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [11]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [12]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [ad2f00f6][81fabc3d-1e44-43b9-9e7c-54fe2b3e11b6], (This step will run and generate new outputs)
Submitted PipelineRun 62b811b8-e77f-45b8-b25c-69b4979f9908
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/62b811b8-e77f-45b8-b25c-69b4979f9908?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-292931/workspaces/quick-starts-ws-292931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [17]:
pipeline_run.wait_for_completion()

PipelineRunId: 62b811b8-e77f-45b8-b25c-69b4979f9908
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/62b811b8-e77f-45b8-b25c-69b4979f9908?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-292931/workspaces/quick-starts-ws-292931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: c090de18-b9f4-4f05-854d-e5441eac626e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c090de18-b9f4-4f05-854d-e5441eac626e?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-292931/workspaces/quick-starts-ws-292931&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
Experiment timeout reached, hence experiment stopped. Current experiment timeout: 1 hour(s) 0 minute(s)
{'runId': 'c090de18-b9f4-4f05-854d-e5441eac626e', 'target': 'automlcluster', 'status':

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [18]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/c090de18-b9f4-4f05-854d-e5441eac626e/metrics_data, 1 files out of an estimated total of 1


In [19]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,c090de18-b9f4-4f05-854d-e5441eac626e_0,c090de18-b9f4-4f05-854d-e5441eac626e_1,c090de18-b9f4-4f05-854d-e5441eac626e_3,c090de18-b9f4-4f05-854d-e5441eac626e_7,c090de18-b9f4-4f05-854d-e5441eac626e_9,c090de18-b9f4-4f05-854d-e5441eac626e_15,c090de18-b9f4-4f05-854d-e5441eac626e_10,c090de18-b9f4-4f05-854d-e5441eac626e_5,c090de18-b9f4-4f05-854d-e5441eac626e_13,c090de18-b9f4-4f05-854d-e5441eac626e_8,...,c090de18-b9f4-4f05-854d-e5441eac626e_68,c090de18-b9f4-4f05-854d-e5441eac626e_235,c090de18-b9f4-4f05-854d-e5441eac626e_265,c090de18-b9f4-4f05-854d-e5441eac626e_281,c090de18-b9f4-4f05-854d-e5441eac626e_236,c090de18-b9f4-4f05-854d-e5441eac626e_251,c090de18-b9f4-4f05-854d-e5441eac626e_269,c090de18-b9f4-4f05-854d-e5441eac626e_253,c090de18-b9f4-4f05-854d-e5441eac626e_270,c090de18-b9f4-4f05-854d-e5441eac626e_254
average_precision_score_weighted,[0.9452854849088579],[0.9430774552761103],[0.9113316400352012],[0.9360938409828844],[0.863441109611488],[0.9412681072479517],[0.9180094755162056],[0.9322543533071588],[0.8918706612001673],[0.9351838499927476],...,[0.934041753142581],[0.9329338382094211],[0.9397572921740808],[0.9462786805722626],[0.9432953427603054],[0.9402647603273322],[0.9421165899662788],[0.9412683656303323],[0.940072260476185],[0.9408556766513062]
recall_score_macro,[0.8586373033716377],[0.8400248639277526],[0.7715503548980092],[0.8309404406671979],[0.5],[0.85293379671514],[0.7830342986805359],[0.8258223157065507],[0.7199042085464307],[0.8286299227807453],...,[0.8209266973347025],[0.8171142599738579],[0.8529986550186834],[0.7431812382444325],[0.84661247921102],[0.8404888138363118],[0.8275640215367441],[0.8320191242045982],[0.8332630307196384],[0.8305609959205547]
average_precision_score_micro,[0.9438108661870281],[0.9518932250847221],[0.8292595991592951],[0.9286666499619728],[0.34688980672362235],[0.927049254973841],[0.8627284771342534],[0.9121381273307443],[0.7745684725039801],[0.9363942179564991],...,[0.9038536802069392],[0.9408447540585734],[0.9397704750360845],[0.9757599735476088],[0.9385582175707273],[0.9238617966293452],[0.9591260627295176],[0.952077778236229],[0.943215912791058],[0.9528078641488802]
average_precision_score_macro,[0.8002241934017709],[0.7935128077764817],[0.7032955249039201],[0.7714243630622724],[0.6008025235108497],[0.7866696845662652],[0.7212070393403528],[0.7615235568533985],[0.6717345425135206],[0.7703127367617799],...,[0.7667370498356081],[0.7590979287560149],[0.7807445089133427],[0.8027034887421737],[0.7933937011724661],[0.7855554263936204],[0.7890244755324982],[0.7867673260011518],[0.7847433186443242],[0.786272727378017]
balanced_accuracy,[0.8586373033716377],[0.8400248639277526],[0.7715503548980092],[0.8309404406671979],[0.5],[0.85293379671514],[0.7830342986805359],[0.8258223157065507],[0.7199042085464307],[0.8286299227807453],...,[0.8209266973347025],[0.8171142599738579],[0.8529986550186834],[0.7431812382444325],[0.84661247921102],[0.8404888138363118],[0.8275640215367441],[0.8320191242045982],[0.8332630307196384],[0.8305609959205547]
AUC_micro,[0.9407563905470939],[0.9497828787698883],[0.8338955102979762],[0.9328061190257428],[0.16153872335963881],[0.9262885698519264],[0.8632228188652029],[0.9102615626664549],[0.7814930436896936],[0.9340371638308248],...,[0.9021996936300999],[0.9395357621621241],[0.937989094937307],[0.9748144165601714],[0.9354968319411847],[0.9213379339025818],[0.9572310103771019],[0.9496635056173357],[0.9410579287931565],[0.9508297534428469]
weighted_accuracy,[0.8517366673467338],[0.8772088568154741],[0.7553980580489636],[0.8611785459590765],[0.017248699294682104],[0.8423515568707629],[0.788920659595202],[0.8273919510055852],[0.7086418238782438],[0.85481207034811],...,[0.8214198937492797],[0.8686809390349665],[0.8607305029804814],[0.9478584359238137],[0.8481784578174532],[0.8324920842042198],[0.8910451192654784],[0.87789987481424],[0.8639184560546207],[0.8805093599623764]
recall_score_weighted,[0.8531623175586024],[0.8695267580716497],[0.7587350729765591],[0.8549314462627157],[0.11698

### Retrieve the Best Model

In [20]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/c090de18-b9f4-4f05-854d-e5441eac626e/model_data, 1 files out of an estimated total of 1


In [21]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/localcomp/code/Users/odl_user_292931/starter_files')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classification_labels=array([...rue, with_mean=False, with_std=False)), ('randomforestclassifier', RandomForestClassifier(n_jobs=1))]))], flatten_transform=False, weights=[0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.13333333333333333, 0.4]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [22]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('61', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.9, eta=0.1, gamma=10, grow_policy='lossguide', max_bin=63, max_depth=10, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=1.875, reg_lambda=1.25, subsample=0.8, tree_method='hist'))], verbose=False)), ('247', Pipeline(memory=None, steps=[('standardscalerwrapper', Stand

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [24]:
#dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
from azureml.core import Workspace, Dataset

subscription_id = 'a0a76bad-11a1-4a2d-9887-97a29122c8ed'
resource_group = 'aml-quickstarts-292931'
workspace_name = 'quick-starts-ws-292931'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_test = Dataset.get_by_name(workspace, name='bankmarketing')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [25]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [26]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,37955,1967
1,199,5090


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [27]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,037c7577-1b69-4749-b9e4-dc2a97e8fd1a,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [28]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [29]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [30]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  fce49f65-048a-48c5-97da-bc60bb460d49
